In [1]:
import pandas as pd
import numpy as np
import requests
from collections import deque
from functools import reduce
import matplotlib.pyplot as plt
#pd.options.display.float_format = '{:,.0f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)

# This notebook outlines the download and formatting process for the HUD residential building permits dataset for counties and places in the GNRC operating region.

Go to this page: https://socds.huduser.gov/permits/  
+ Under "Main Criteria" select "States and Counties"  
+ Under "Periodicity" select "Annual"  
+ Under "Select State(s)" select "Tennessee", then select "Show Counties and Jurisdictions for Selected States"  
+ Under "Select Counties", highlight all counties, do the same for all jurisdictions under "Select Permitting Jurisdictions"  
+ Under "Select Year(s), select the most recent available year  
+ Under "Select Series", select "All Permits", "Single Family", and "All Multifamily"  
+ Finally, select the "Get Data" button  
This will populate a page with the information in table format. Scroll to the bottom and download in excel or csv format. 

Save as a csv in the "Data Downloads" folder and import as downloaded.

In [2]:
data = pd.read_csv('../../Data Downloads/HUD Building Permits.csv')

In [3]:
data.head()

,Location,Year,Series,Series Code,Permits
0,Anderson County,2021,Total Units,1,458
1,Anderson County,2021,Units in Single-Family Structures,2,261
2,Anderson County,2021,Units in All Multi-Family Structures,3,197
3,Bedford County,2021,Total Units,1,390
4,Bedford County,2021,Units in Single-Family Structures,2,390


When you examine the data download in excel, you can see that the headers repeat every few records, delete these records. 

In [4]:
data = data.loc[data['Location'] != 'Location']

In [5]:
#strip extra spaces
data['Location'] = data['Location'].str.strip()

In [6]:
data.head(100)

,Location,Year,Series,Series Code,Permits
0,Anderson County,2021,Total Units,1,458
1,Anderson County,2021,Units in Single-Family Structures,2,261
2,Anderson County,2021,Units in All Multi-Family Structures,3,197
3,Bedford County,2021,Total Units,1,390
4,Bedford County,2021,Units in Single-Family Structures,2,390
5,Bedford County,2021,Units in All Multi-Family Structures,3,0
6,Benton County,2021,Total Units,1,4
7,Benton County,2021,Units in Single-Family Structures,2,4
8,Benton County,2021,Units in All Multi-Family Structures,3,0
9,Blount County,2021,Total Units,1,922


We have three different target geographies that are downloaded: counties, unincorporated areas, and places. We can split these up to concatenate later as the data cleaning processes will be different. For now, we are only taking geographies within the GNRC 14 county operating region. You can see from the download that all counties contain the string "County", and that unincorporated areas contain the string "COUNTY".

In [7]:
counties = data[data['Location'].str.contains('County')]

In [8]:
counties.tail(100)

,Location,Year,Series,Series Code,Permits
180,Meigs County,2021,Units in All Multi-Family Structures,3,0
182,Monroe County,2021,Total Units,1,215
183,Monroe County,2021,Units in Single-Family Structures,2,215
184,Monroe County,2021,Units in All Multi-Family Structures,3,0
185,Montgomery County,2021,Total Units,1,4008
186,Montgomery County,2021,Units in Single-Family Structures,2,2119
187,Montgomery County,2021,Units in All Multi-Family Structures,3,1889
188,Moore County,2021,Total Units,1,30
189,Moore County,2021,Units in Single-Family Structures,2,30
190,Moore County,2021,Units in All Multi-Family Structures,3,0


In [9]:
region = ['Cheatham County', 'Davidson County', 'Dickson County', 'Houston County', 'Humphreys County', 'Maury County', 'Montgomery County', 
          'Robertson County', 'Rutherford County', 'Stewart County', 'Sumner County', 'Trousdale County', 'Williamson County', 'Wilson County']

In [10]:
counties = counties.loc[counties['Location'].isin(region)]

In [11]:
counties['Location'] = counties['Location'] + ", Tennessee"

In [12]:
counties.head(100)

,Location,Year,Series,Series Code,Permits
27,"Cheatham County, Tennessee",2021,Total Units,1,407
28,"Cheatham County, Tennessee",2021,Units in Single-Family Structures,2,260
29,"Cheatham County, Tennessee",2021,Units in All Multi-Family Structures,3,147
51,"Davidson County, Tennessee",2021,Total Units,1,16310
52,"Davidson County, Tennessee",2021,Units in Single-Family Structures,2,4105
53,"Davidson County, Tennessee",2021,Units in All Multi-Family Structures,3,12205
60,"Dickson County, Tennessee",2021,Total Units,1,617
61,"Dickson County, Tennessee",2021,Units in Single-Family Structures,2,473
62,"Dickson County, Tennessee",2021,Units in All Multi-Family Structures,3,144
121,"Houston County, Tennessee",2021,Total Units,1,1


Counties are ready to go, now for Unincorporated Areas

In [13]:
unincorporated = data[data['Location'].str.contains('COUNTY')]
unincorporated.head()

,Location,Year,Series,Series Code,Permits
303,ANDERSON COUNTY UNINCORPORATED AREA,2021,Total Units,1,83
304,ANDERSON COUNTY UNINCORPORATED AREA,2021,Units in Single-Family Structures,2,83
305,ANDERSON COUNTY UNINCORPORATED AREA,2021,Units in All Multi-Family Structures,3,0
339,BEDFORD COUNTY UNINCORPORATED AREA,2021,Total Units,1,173
340,BEDFORD COUNTY UNINCORPORATED AREA,2021,Units in Single-Family Structures,2,173


In [14]:
region = ['CHEATHAM COUNTY UNINCORPORATED AREA', 'DAVIDSON COUNTY UNINCORPORATED AREA', 'DICKSON COUNTY UNINCORPORATED AREA', 
          'HOUSTON COUNTY UNINCORPORATED AREA', 'HUMPHREYS COUNTY UNINCORPORATED AREA', 'MAURY COUNTY UNINCORPORATED AREA', 
          'MONTGOMERY COUNTY UNINCORPORATED AREA', 'ROBERTSON COUNTY UNINCORPORATED AREA', 'RUTHERFORD COUNTY UNINCORPORATED AREA', 
          'STEWART COUNTY UNINCORPORATED AREA', 'SUMNER COUNTY UNINCORPORATED AREA', 'TROUSDALE COUNTY UNINCORPORATED AREA', 
          'WILLIAMSON COUNTY UNINCORPORATED AREA', 'WILSON COUNTY UNINCORPORATED AREA']

#Davidson, Houston, Humphreys nor Stewart have unincorporated lines

In [15]:
unincorporated = unincorporated.loc[unincorporated['Location'].isin(region)]

In [16]:
unincorporated['Location'] = unincorporated['Location'].replace({'CHEATHAM COUNTY UNINCORPORATED AREA': 'Cheatham Unincorporated', 
                                                                 'DICKSON COUNTY UNINCORPORATED AREA': "Dickson Unincorporated", 
                                                                 'MAURY COUNTY UNINCORPORATED AREA': 'Maury Unincorporated', 
                                                                 'MONTGOMERY COUNTY UNINCORPORATED AREA': 'Montgomery Unincorporated', 
                                                                 'ROBERTSON COUNTY UNINCORPORATED AREA': 'Robertson Unincorporated', 
                                                                 'RUTHERFORD COUNTY UNINCORPORATED AREA': 'Rutherford Unincorporated',
                                                                 'SUMNER COUNTY UNINCORPORATED AREA': 'Sumner Unincorporated', 
                                                                 'TROUSDALE COUNTY UNINCORPORATED AREA': 'Trousdale Unincorporated', 
                                                                 'WILLIAMSON COUNTY UNINCORPORATED AREA': 'Williamson Unincorporated', 
                                                                 'WILSON COUNTY UNINCORPORATED AREA': 'Wilson Unincorporated'})

In [17]:
unincorporated.head()

,Location,Year,Series,Series Code,Permits
435,Cheatham Unincorporated,2021,Total Units,1,100
436,Cheatham Unincorporated,2021,Units in Single-Family Structures,2,100
437,Cheatham Unincorporated,2021,Units in All Multi-Family Structures,3,0
528,Dickson Unincorporated,2021,Total Units,1,290
529,Dickson Unincorporated,2021,Units in Single-Family Structures,2,290


Places are going to be more difficult, we're looking for the following: 
Adams city, Tennessee: Robertson  
Ashland City town, Tennessee: Cheatham  
Belle Meade city, Tennessee: Davidson  
Berry Hill city, Tennessee: Davidson  
Brentwood city, Tennessee: Williamson  
Burns town, Tennessee: Dickson  
Cedar Hill city, Tennessee: Robertson  
Charlotte town, Tennessee: Dickson  
Clarksville city, Tennessee: Montgomery  
Columbia city, Tennessee: Maury  
Coopertown town, Tennessee: Robertson  
Cross Plains city, Tennessee: Robertson  
Cumberland City town, Tennessee: Stewart  
Dickson city, Tennessee: Dickson  
Dover city, Tennessee: Stewart  
Eagleville city, Tennessee: Rutherford  
Erin city, Tennessee: Houston  
Fairview city, Tennessee: Williamson  
Forest Hills city, Tennessee: Davidson  
Franklin city, Tennessee: Williamson  
Gallatin city, Tennessee: Sumner  
Goodlettsville city, Tennessee: Davidson/Sumner  
Greenbrier town, Tennessee: Robertson  
Hendersonville city, Tennessee: Sumner  
Kingston Springs town, Tennessee: Cheatham  
La Vergne city, Tennessee: Rutherford  
Lafayette city, Tennessee: Macon  
Lebanon city, Tennessee: Wilson  
McEwen city, Tennessee: Humphreys  
Millersville city, Tennessee: Robertson/Sumner  
Mitchellville city, Tennessee: Sumner  
Mount Juliet city, Tennessee: Wilson  
Mount Pleasant city, Tennessee: Maury  
Murfreesboro city, Tennessee: Rutherford  
Nashville-Davidson metropolitan government (balance): Davidson  
New Johnsonville city, Tennessee: Humphreys  
Nolensville town, Tennessee: Williamson  
Oak Hill city, Tennessee: Davidson  
Pegram town, Tennessee: Cheatham  
Pleasant View city, Tennessee: Cheatham  
Portland city, Tennessee: Robertson/Sumner  
Ridgetop city, Tennessee: Davidson/Robertson  
Slayden town, Tennessee: Dickson  
Smyrna town, Tennessee: Rutherford  
Spring Hill city, Tennessee: Maury/Williamson  
Springfield city, Tennessee: Robertson  
Tennessee Ridge town, Tennessee: Houston/Stewart  
Thompson's Station town, Tennessee: Williamson  
Vanleer town, Tennessee: Dickson  
Watertown city, Tennessee: Wilson  
Waverly city, Tennessee: Humphreys  
Westmoreland town, Tennessee: Sumner  
White Bluff town, Tennessee: Dickson  
White House city, Tennessee: Robertson/Sumner    

The downloaded data is in all caps, and doesn't have the ", Tennessee" of the "town.. city... etc.". There are 54 records. We can reverse index into the geographies that are *not* counties or unincorporated areas, reformat the capital letters, and then I'll make a list of only the first word of the place to run through and see how close we can get that way.


In [18]:
place = data[~data['Location'].str.contains('County')]
place = place[~place['Location'].str.contains('COUNTY')]
place.head()

,Location,Year,Series,Series Code,Permits
282,ADAMS,2021,Total Units,1,0
283,ADAMS,2021,Units in Single-Family Structures,2,0
284,ADAMS,2021,Units in All Multi-Family Structures,3,0
285,ADAMSVILLE TOWN,2021,Total Units,1,1
286,ADAMSVILLE TOWN,2021,Units in Single-Family Structures,2,1


In [19]:
place['Location'] = place['Location'].str.title()
place.head(50)

,Location,Year,Series,Series Code,Permits
282,Adams,2021,Total Units,1,0
283,Adams,2021,Units in Single-Family Structures,2,0
284,Adams,2021,Units in All Multi-Family Structures,3,0
285,Adamsville Town,2021,Total Units,1,1
286,Adamsville Town,2021,Units in Single-Family Structures,2,1
287,Adamsville Town,2021,Units in All Multi-Family Structures,3,0
288,Alamo Town,2021,Total Units,1,0
289,Alamo Town,2021,Units in Single-Family Structures,2,0
290,Alamo Town,2021,Units in All Multi-Family Structures,3,0
291,Alcoa,2021,Total Units,1,192


In [31]:
#I've gone through the excel document to see what these are named - hopefully it is consistent year to year so that this saves time.
places = ["Adams", 
          "Ashland City Town", 
          "Belle Meade", 
          "Berry Hill", 
          "Brentwood", 
          "Burns Town",  
          "Charlotte Town", 
          "Clarksville", 
          "Columbia", 
          "Coopertown Town", 
          "Cross Plains", 
          "Cumberland City Town", 
          "Dickson", 
          "Dover", 
          "Eagleville", 
          "Erin", 
          "Fairview", 
          "Forest Hills", 
          "Franklin", 
          "Gallatin", 
          "Goodlettsville", 
          "Greenbrier Town",
          "Hendersonville", 
          "Kingston Springs Town", 
          "La Vergne", 
          "Lafayette", 
          "Lebanon", 
          "Mcewen", 
          "Millersville", 
          "Mitchellville Town", 
          "Mount Juliet", 
          "Mount Pleasant", 
          "Murfreesboro", 
          "Nashville-Davidson", 
          "New Johnsonville", 
          "Nolensville Town", 
          "Oak Hill", 
          "Pegram Town", 
          "Pleasant View", 
          "Portland", 
          "Ridgetop Town",
          "Smyrna Town", 
          "Spring Hill Town", 
          "Springfield", 
          "Tennessee Ridge Town", 
          "Thompsons Station Town",
          "Watertown", 
          "Waverly", 
          "Westmoreland Town", 
          "White Bluff Town", 
          "White House"]

In [34]:
place = place.loc[place['Location'].isin(places)].reset_index(drop = True)

In [35]:
place.head()

,Location,Year,Series,Series Code,Permits
0,Adams,2021,Total Units,1,0
1,Adams,2021,Units in Single-Family Structures,2,0
2,Adams,2021,Units in All Multi-Family Structures,3,0
3,Ashland City Town,2021,Total Units,1,193
4,Ashland City Town,2021,Units in Single-Family Structures,2,46


In [36]:
transp = place.set_index('Location').transpose()

In [37]:
transp.head()

Location,Adams,Adams,Adams,Ashland City Town,Ashland City Town,Ashland City Town,Belle Meade,Belle Meade,Belle Meade,Berry Hill,Berry Hill,Berry Hill,Brentwood,Brentwood,Brentwood,Burns Town,Burns Town,Burns Town,Charlotte Town,Charlotte Town,Charlotte Town,Clarksville,Clarksville,Clarksville,Columbia,Columbia,Columbia,Coopertown Town,Coopertown Town,Coopertown Town,Cross Plains,Cross Plains,Cross Plains,Cumberland City Town,Cumberland City Town,Cumberland City Town,Dickson,Dickson,Dickson,Dover,Dover,Dover,Eagleville,Eagleville,Eagleville,Erin,Erin,Erin,Fairview,Fairview,Fairview,Forest Hills,Forest Hills,Forest Hills,Franklin,Franklin,Franklin,Gallatin,Gallatin,Gallatin,Goodlettsville,Goodlettsville,Goodlettsville,Greenbrier Town,Greenbrier Town,Greenbrier Town,Hendersonville,Hendersonville,Hendersonville,Kingston Springs Town,Kingston Springs Town,Kingston Springs Town,La Vergne,La Vergne,La Vergne,Lafayette,Lafayette,Lafayette,Lebanon,Lebanon,Lebanon,Mcewen,Mcewen,Mcewen,Millersville,Millersville,Millersville,Mitchellville Town,Mitchellville Town,Mitchellville Town,Mount Juliet,Mount Juliet,Mount Juliet,Mount Pleasant,Mount Pleasant,Mount Pleasant,Murfreesboro,Murfreesboro,Murfreesboro,Nashville-Davidson,Nashville-Davidson,Nashville-Davidson,New Johnsonville,New Johnsonville,New Johnsonville,Nolensville Town,Nolensville Town,Nolensville Town,Oak Hill,Oak Hill,Oak Hill,Pegram Town,Pegram Town,Pegram Town,Pleasant View,Pleasant View,Pleasant View,Portland,Portland,Portland,Ridgetop Town,Ridgetop Town,Ridgetop Town,Smyrna Town,Smyrna Town,Smyrna Town,Spring Hill Town,Spring Hill Town,Spring Hill Town,Springfield,Springfield,Springfield,Tennessee Ridge Town,Tennessee Ridge Town,Tennessee Ridge Town,Thompsons Station Town,Thompsons Station Town,Thompsons Station Town,Watertown,Watertown,Watertown,Waverly,Waverly,Waverly,Westmoreland Town,Westmoreland Town,Westmoreland Town,White Bluff Town,White Bluff Town,White Bluff Town,White House,White House,White House
Year,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021
Series,Total Units,Units in Single-Family Structures,Units in All Multi-Family Structures,Total Units,Units in Single-Family Structures,Units in All Multi-Family Structures,Total Units,Units in Single-Family Structures,Units in All Multi-Family Structures,Total Units,Units in Single-Family Structures,Units in All Multi-Family Structures,Total Units,Units in Single-Family Structures,Units in All Multi-Family Structures,Total Units,Units in Single-Family Structures,Units in All Multi-Family Structures,Total Units,Units in Single-Family Structures,Units in All Multi-Family Structures,Total Units,Units in Single-Family Structures,Units in All Multi-Family Structures,Total Units,Units in Single-Family Structures,Units in All Multi-Family Structures,Total Units,Units in Single-Family Structures,Units in All Multi-Family Structures,Total Units,Units in Single-Family Structures,Units in All Multi-Family Structures,Total Units,Units in Single-Family Structures,Units in All Multi-Family Structures,Total Units,Units in Single-Family Structures,Units in All Multi-Family Structures,Total Units,Units in Single-Family Structures,Units in All Multi-Family Structures,Total Units,Units in Single-Family Structures,Units in All Multi-Fami

In [38]:
transp = transp.rename(columns = {"Adams": 'Adams city, Tennessee', "Ashland City Town":'Ashland City town, Tennessee',
                                  "Belle Meade": 'Belle Meade city, Tennessee', "Berry Hill": 'Berry Hill city, Tennessee', 
                                  "Brentwood": 'Brentwood city, Tennessee', "Burns Town": 'Burns town, Tennessee',  
                                  "Charlotte Town": 'Charlotte town, Tennessee',  "Clarksville": 'Clarksville city, Tennessee', 
                                  "Columbia": 'Columbia city, Tennessee', "Coopertown Town": 'Coopertown town, Tennessee', 
                                  "Cross Plains": 'Cross Plains city, Tennessee', "Cumberland City Town":'Cumberland City town, Tennessee', 
                                  "Dickson": 'Dickson city, Tennessee', "Dover": 'Dover city, Tennessee', 
                                  "Eagleville": 'Eagleville city, Tennessee', "Erin": 'Erin city, Tennessee', 
                                  "Fairview": 'Fairview city, Tennessee', "Forest Hills": 'Forest Hills city, Tennessee', 
                                  "Franklin": 'Franklin city, Tennessee', "Gallatin": 'Gallatin city, Tennessee', 
                                  "Goodlettsville": 'Goodlettsville city, Tennessee', "Greenbrier Town": 'Greenbrier town, Tennessee',
                                  "Hendersonville": 'Hendersonville city, Tennessee', "Kingston Springs Town": 'Kingston Springs town, Tennessee', 
                                  "La Vergne": 'La Vergne city, Tennessee', "Lafayette": 'La Vergne city, Tennessee', 
                                  "Lebanon": 'Lebanon city, Tennessee', "Mcewen": 'McEwen city, Tennessee', 
                                  "Millersville": 'Millersville city, Tennessee', "Mitchellville Town": 'Mitchellville city, Tennessee', 
                                  "Mount Juliet": 'Mount Juliet city, Tennessee', "Mount Pleasant": 'Mount Pleasant city, Tennessee', 
                                  "Murfreesboro": 'Murfreesboro city, Tennessee', "Nashville-Davidson": "Nashville-Davidson metropolitan government (balance)", 
                                  "New Johnsonville": 'New Johnsonville city, Tennessee', "Nolensville Town": 'Nolensville town, Tennessee', 
                                  "Oak Hill": 'Oak Hill city, Tennessee', "Pegram Town": 'Pegram town, Tennessee', 
                                  "Pleasant View": 'Pleasant View city, Tennessee', "Portland": 'Portland city, Tennessee', 
                                  "Ridgetop Town": 'Ridgetop city, Tennessee',"Smyrna Town": 'Smyrna town, Tennessee', 
                                  "Spring Hill Town": 'Spring Hill city, Tennessee', "Springfield": 'Springfield city, Tennessee', 
                                  "Tennessee Ridge Town": 'Tennessee Ridge town, Tennessee', "Thompsons Station Town": "Thompson's Station town, Tennessee",
                                  "Watertown": 'Watertown city, Tennessee', "Waverly": 'Waverly city, Tennessee', 
                                  "Westmoreland Town": 'Westmoreland town, Tennessee', "White Bluff Town": 'White Bluff town, Tennessee', 
                                  "White House": 'White House city, Tennessee'})

In [39]:
place = transp.transpose()

In [45]:
place = place.reset_index()

In [46]:
dfs = [counties, unincorporated, place]
df = pd.concat(dfs)

In [52]:
df = df.rename(columns = {'Location':'NAME'})

In [54]:
geos = pd.read_csv('../../Data Downloads/geofips.csv')

In [55]:
geos.head()

,NAME,GEO_ID
0,"Cheatham County, Tennessee",0500000US47021
1,"Davidson County, Tennessee",0500000US47037
2,"Dickson County, Tennessee",0500000US47043
3,"Houston County, Tennessee",0500000US47083
4,"Humphreys County, Tennessee",0500000US47085


In [56]:
df = df.merge(geos, how = 'inner')

In [58]:
df.to_csv('../../Outputs/HUDSOC_ResidentialPermits2021.csv', index = False)